_Lambda School Data Science — Linear Models_

# Intro to Predictive Modeling

### Objectives
- recognize examples of supervised learning with tabular data
- distinguish between regression problems and classification problems
- explain why overfitting is a problem and model validation is important
- do train/test split
- begin with baselines for regression

I like Brandon Rohrer’s blog post, [“What questions can machine learning answer?”](https://brohrer.github.io/five_questions_data_science_answers.html)

We’ll focus on two of these questions in Unit 2. These are both types of “supervised learning.”

- “Is this A or B?” (Classification)
- “How Much / How Many?” (Regression)

**This unit, you’ll do four supervised learning projects** with “tabular data” (data in tables, like spreadsheets).

- Predict New York City apartment rents <-- **Today, we'll start this project!**
- Predict which water pumps in Tanzania need repairs
- Predict the prices suppliers will quote Caterpillar for industrial parts
- Choose your own labeled, tabular dataset, train a predictive model, and publish a blog post or web app with visualizations to explain your model!

# Predict NYC apartment rent 🏠💸

You'll use a real-world data with rent prices for a subset of apartments in New York City!

Run this code cell to load the dataset: 

In [1]:
import pandas as pd

LOCAL = '../data/nyc/nyc-rent-2016.csv'
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/nyc/nyc-rent-2016.csv'

df = pd.read_csv(WEB)
assert df.shape == (48300, 34)

### Install [pandas-profiling](https://github.com/pandas-profiling/pandas-profiling), version >= 2

In [2]:
!pip install --upgrade pandas-profiling

Requirement already up-to-date: pandas-profiling in /Users/ashwin/anaconda3/lib/python3.7/site-packages (2.1.0)


In [3]:
import pandas_profiling
pandas_profiling.__version__

'2.1.0'

## Define the problem
- Is this **supervised** learning?
- Is this **tabular** data?
- Is this **regression** or **classification**?

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48300 entries, 0 to 48299
Data columns (total 34 columns):
bathrooms               48300 non-null float64
bedrooms                48300 non-null int64
created                 48300 non-null object
description             46879 non-null object
display_address         48168 non-null object
latitude                48300 non-null float64
longitude               48300 non-null float64
price                   48300 non-null int64
street_address          48290 non-null object
interest_level          48300 non-null object
elevator                48300 non-null int64
cats_allowed            48300 non-null int64
hardwood_floors         48300 non-null int64
dogs_allowed            48300 non-null int64
doorman                 48300 non-null int64
dishwasher              48300 non-null int64
no_fee                  48300 non-null int64
laundry_in_building     48300 non-null int64
fitness_center          48300 non-null int64
pre-war                 4

In [5]:
df.describe()

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,...,high_speed_internet,balcony,swimming_pool,new_construction,exclusive,terrace,loft,garden_patio,common_outdoor_space,wheelchair_access
count,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,...,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000,48300.000000
mean,1.178313,1.508799,40.750782,-73.972365,3438.297950,0.520393,0.477702,0.474576,0.446832,0.417805,...,0.085839,0.058509,0.053002,0.050476,0.043623,0.043851,0.041263,0.037992,0.026356,0.026066
std,0.426120,1.092232,0.039560,0.029563,1401.422247,0.499589,0.499508,0.499358,0.497170,0.493203,...,0.280128,0.234706,0.224040,0.218928,0.204257,0.204765,0.198900,0.191178,0.160194,0.159334
min,0.000000,0.000000,40.571200,-74.094000,1025.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,40.728100,-73.991700,2495.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,40.751600,-73.977900,3100.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,2.000000,40.774000,-73.954700,4000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10.000000,8.000000,40.915400,-73.700100,9999.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
df.sample(n=10)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,high_speed_internet,balcony,swimming_pool,new_construction,exclusive,terrace,loft,garden_patio,common_outdoor_space,wheelchair_access
9504,1.0,2,2016-04-03 03:42:59,Now available this spacious Flex 2 bedrooms 1 ...,3 Ave.,40.7823,-73.9510,3800,1641 3 Ave.,low,...,1,0,1,0,0,1,0,0,0,0
34407,1.0,2,2016-05-07 03:53:31,TAKE ADVANTAGE OF THIS GENEROUS SIZED STUDIO A...,Gold St.,40.7088,-74.0059,2700,33 Gold St.,low,...,0,0,0,0,0,0,1,0,0,0
4936,1.0,0,2016-06-02 03:10:55,***Large Studio in the Upper East Side*** -P/T...,East 86th Street,40.7771,-73.9505,2450,336 East 86th Street,medium,...,0,0,0,0,0,0,0,0,0,0
20447,1.0,0,2016-06-11 04:21:50,Current Specials: <br />Waived Applica...,95 Wall Street,40.7047,-74.0076,3778,95 Wall Street,low,...,0,0,0,0,0,0,0,0,0,0
12927,1.0,1,2016-06-29 04:58:28,NaN,14th Street,40.6633,-73.9826,2050,411 14th Street,low,...,0,0,0,0,1,0,0,0,0,0
38730,2.0,4,2016-05-13 06:08:04,residents are surrounded by restaurants on eve...,E 91st St.,40.7801,-73.9475,5400,354 E 91st St.,low,...,1,1,0,0,0,0,0,0,0,0
18480,1.0,1,2016-06-16 06:58:07,"Gas, heat and hot Water included in rent- Live...",34th Avenue,40.7676,-73.8198,1750,147-11 34th Avenue,low,...,0,0,0,0,0,0,0,0,0,0
30726,1.0,3,2016-05-13 05:44:32,Gorgeous Pre-War building Large one bedroom wi...,East 47th Street,40.7542,-73.9724,3600,155 East 47th Street,low,...,0,0,0,0,0,0,0,0,0,0
37532,1.0,2,2016-05-14 02:12:25,A detailed description is coming soon. Please ...,Buffalo Ave,40.6781,-73.9245,1820,29 Buffalo Ave,low,...,0,0,0,0,0,0,0,0,0,0
6199,1.5,2,2016-04-19 03:15:22,"A renter's dream, every room is spacious and c...",231 Marion Street,40.6809,-73.9204,1850,231 Marion Street,low,...,0,0,0,0,0,0,0,0,0,0


In [7]:
profile = df.profile_report()
profile.to_file(output_file="output.html")

## Explain why overfitting is a problem and model validation is important

#### Jason Brownlee, [Overfitting and Underfitting With Machine Learning Algorithms](https://machinelearningmastery.com/overfitting-and-underfitting-with-machine-learning-algorithms/)

> The goal of a good machine learning model is to **generalize** well from the training data to any data from the problem domain. This allows us to make predictions in the future on data the model has never seen.

> The cause of poor performance in machine learning is either overfitting or underfitting the data.

> **Overfitting** refers to a model that models the training data too well. Overfitting happens when a model learns the detail and noise in the training data to the extent that it negatively impacts the performance of the model on new data. 

> **Underfitting** refers to a model that can neither model the training data nor generalize to new data.

> Ideally, you want to select a model at the sweet spot between underfitting and overfitting.


#### Rob Hyndman & George Athanasopoulos, [_Forecasting: Principles and Practice_, Chapter 3.4](https://otexts.com/fpp2/accuracy.html), Evaluating forecast accuracy:

> The following points should be noted.

> - A model which fits the training data well will not necessarily forecast well.
> - A perfect fit can always be obtained by using a model with enough parameters.
> - Over-fitting a model to data is just as bad as failing to identify a systematic pattern in the data.

> **The accuracy of forecasts can only be determined by considering how well a model performs on new data that were not used when fitting the model.**

> When choosing models, it is common practice to separate the available data into two portions, training and test data, where the training data is used to estimate any parameters of a forecasting method and the test data is used to evaluate its accuracy. Because the test data is not used in determining the forecasts, it should provide a reliable indication of how well the model is likely to forecast on new data.

![](https://otexts.com/fpp2/fpp_files/figure-html/traintest-1.png)

> The size of the test set is typically about 20% of the total sample, although this value depends on how long the sample is and how far ahead you want to forecast. The test set should ideally be at least as large as the maximum forecast horizon required.

> Some references describe the test set as the “hold-out set” because these data are “held out” of the data used for fitting. Other references call the training set the “in-sample data” and the test set the “out-of-sample data”. We prefer to use “training data” and “test data” in this book.


#### Rachel Thomas, [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/)

> An all-too-common scenario: a seemingly impressive machine learning model is a complete failure when implemented in production. The fallout includes leaders who are now skeptical of machine learning and reluctant to try it again. How can this happen?

> One of the most likely culprits for this disconnect between results in development vs results in production is a poorly chosen validation set (or even worse, no validation set at all). 



#### James, Witten, Hastie, Tibshirani, [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/), Chapter 2.2, Assessing Model Accuracy

> In general, we do not really care how well the method works training on the training data. Rather, _we are interested in the accuracy of the predictions that we obtain when we apply our method to previously unseen test data._ Why is this what we care about? 

> Suppose that we are interested test data in developing an algorithm to predict a stock’s price based on previous stock returns. We can train the method using stock returns from the past 6 months. But we don’t really care how well our method predicts last week’s stock price. We instead care about how well it will predict tomorrow’s price or next month’s price. 

> On a similar note, suppose that we have clinical measurements (e.g. weight, blood pressure, height, age, family history of disease) for a number of patients, as well as information about whether each patient has diabetes. We can use these patients to train a statistical learning method to predict risk of diabetes based on clinical measurements. In practice, we want this method to accurately predict diabetes risk for _future patients_ based on their clinical measurements. We are not very interested in whether or not the method accurately predicts diabetes risk for patients used to train the model, since we already know which of those patients have diabetes.

#### Owen Zhang, [Winning Data Science Competitions](https://www.slideshare.net/OwenZhang2/tips-for-data-science-competitions/8)

> Good validation is _more important_ than good models. 

## Do train/test split

We have two options for where we choose to split:
- Time
- Random




This choice depends on your goals. Rachel Thomas explains why you may want to split on time:

#### Rachel Thomas, [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/)

> If your data is a time series, choosing a random subset of the data will be both too easy (you can look at the data both before and after the dates your are trying to predict) and not representative of most business use cases (where you are using historical data to build a model for use in the future). If your data includes the date and you are building a model to use in the future, you will want to choose a continuous section with the latest dates as your validation set


For this project, we'll split based on time. 

- Use data from April & May 2016 to train.
- Use data from June 2016 to test.

(But in some future projects this unit, we'll do random splits, and explain why.)

In [8]:
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)
df['created']

0       2016-06-24 07:54:24
1       2016-06-12 12:19:27
2       2016-04-17 03:26:41
3       2016-04-18 02:22:02
4       2016-04-28 01:32:41
5       2016-04-19 04:24:47
6       2016-04-27 03:19:56
7       2016-04-13 06:01:42
8       2016-04-20 02:36:35
9       2016-04-02 02:58:15
10      2016-04-14 01:10:30
11      2016-06-03 03:21:22
12      2016-04-19 05:37:25
13      2016-04-09 01:22:11
14      2016-06-01 03:11:01
15      2016-04-18 02:36:00
16      2016-04-22 05:57:16
17      2016-04-19 03:36:28
18      2016-04-20 04:00:32
19      2016-04-09 03:49:42
20      2016-04-12 02:39:45
21      2016-04-07 02:11:45
22      2016-04-13 05:17:43
23      2016-04-17 02:26:52
24      2016-06-07 04:39:56
25      2016-04-27 06:43:40
26      2016-04-25 02:29:50
27      2016-04-22 05:39:10
28      2016-04-21 05:27:10
29      2016-04-26 02:59:37
                ...        
48270   2016-06-02 13:24:18
48271   2016-04-25 01:13:37
48272   2016-04-26 04:43:14
48273   2016-04-08 05:51:57
48274   2016-04-29 1

In [9]:
df['created'].describe()

count                   48300
unique                  47643
top       2016-06-12 13:20:45
freq                        3
first     2016-04-01 22:12:41
last      2016-06-29 21:41:47
Name: created, dtype: object

In [10]:
df['month'] = df['created'].dt.month

In [11]:
df['month'].describe()

count    48300.000000
mean         5.014928
std          0.824557
min          4.000000
25%          4.000000
50%          5.000000
75%          6.000000
max          6.000000
Name: month, dtype: float64

In [16]:
train = df[df['month'] < 6]
test = df[df['month'] == 6]
assert train.shape[0] + test.shape[0] == df.shape[0]

In [17]:
train['created'].describe()

count                   31515
unique                  31116
top       2016-05-02 03:41:36
freq                        3
first     2016-04-01 22:12:41
last      2016-05-31 23:10:48
Name: created, dtype: object

In [18]:
test['created'].describe()

count                   16785
unique                  16527
top       2016-06-05 01:32:36
freq                        3
first     2016-06-01 01:10:37
last      2016-06-29 21:41:47
Name: created, dtype: object

## Begin with baselines for regression

### Why begin with baselines?

[My mentor](https://www.linkedin.com/in/jason-sanchez-62093847/) [taught me](https://youtu.be/0GrciaGYzV0?t=40s):

>***Your first goal should always, always, always be getting a generalized prediction as fast as possible.*** You shouldn't spend a lot of time trying to tune your model, trying to add features, trying to engineer features, until you've actually gotten one prediction, at least. 

> The reason why that's a really good thing is because then ***you'll set a benchmark*** for yourself, and you'll be able to directly see how much effort you put in translates to a better prediction. 

> What you'll find by working on many models: some effort you put in, actually has very little effect on how well your final model does at predicting new observations. Whereas some very easy changes actually have a lot of effect. And so you get better at allocating your time more effectively.

My mentor's advice is echoed and elaborated in several sources:

[Always start with a stupid model, no exceptions](https://blog.insightdatascience.com/always-start-with-a-stupid-model-no-exceptions-3a22314b9aaa)

> Why start with a baseline? A baseline will take you less than 1/10th of the time, and could provide up to 90% of the results. A baseline puts a more complex model into context. Baselines are easy to deploy.

[Measure Once, Cut Twice: Moving Towards Iteration in Data Science](https://blog.datarobot.com/measure-once-cut-twice-moving-towards-iteration-in-data-science)

> The iterative approach in data science starts with emphasizing the importance of getting to a first model quickly, rather than starting with the variables and features. Once the first model is built, the work then steadily focuses on continual improvement.

[*Data Science for Business*](https://books.google.com/books?id=4ZctAAAAQBAJ&pg=PT276), Chapter 7.3: Evaluation, Baseline Performance, and Implications for Investments in Data

> *Consider carefully what would be a reasonable baseline against which to compare model performance.* This is important for the data science team in order to understand whether they indeed are improving performance, and is equally important for demonstrating to stakeholders that mining the data has added value.

### What does baseline mean?

Baseline is an overloaded term, as you can see in the links above. Baseline has multiple meanings:

#### The score you'd get by guessing

> A baseline for classification can be the most common class in the training dataset.

> A baseline for regression can be the mean of the training labels. 

> A baseline for time-series regressions can be the value from the previous timestep. —[Will Koehrsen](https://twitter.com/koehrsen_will/status/1088863527778111488)

#### Fast, first models that beat guessing

What my mentor was talking about.

#### Complete, tuned "simpler" model

Can be simpler mathematically and computationally. For example, Logistic Regression versus Deep Learning.

Or can be simpler for the data scientist, with less work. For example, a model with less feature engineering versus a model with more feature engineering.

#### Minimum performance that "matters"

To go to production and get business value.

#### Human-level performance 

Your goal may to be match, or nearly match, human performance, but with better speed, cost, or consistency.

Or your goal may to be exceed human performance.

In [20]:
train['price'].mean()

3432.7534190068222

In [23]:
first10 = test[['price']].head(10)
first10

,price
0,3000
1,5465
11,1675
14,3050
24,7400
34,2300
39,2396
47,1900
58,2000
81,2625


In [24]:
first10['predicted'] = [3432]*10
first10

,price,predicted
0,3000,3432
1,5465,3432
11,1675,3432
14,3050,3432
24,7400,3432
34,2300,3432
39,2396,3432
47,1900,3432
58,2000,3432
81,2625,3432


In [25]:
first10['error'] = first10['price'] - first10['predicted']
first10

,price,predicted,error
0,3000,3432,-432
1,5465,3432,2033
11,1675,3432,-1757
14,3050,3432,-382
24,7400,3432,3968
34,2300,3432,-1132
39,2396,3432,-1036
47,1900,3432,-1532
58,2000,3432,-1432
81,2625,3432,-807


In [26]:
first10['error'].abs().mean()

1451.1

In [27]:
from sklearn.metrics import mean_absolute_error

In [28]:
mean_absolute_error(first10['price'], first10['predicted'])

1451.1

In [32]:
y_test = test['price']
y_pred = [train['price'].mean()] * len(y_test)
print(len(y_test))
print(len(y_pred))

16785
16785


In [33]:
mean_absolute_error(y_pred, y_test)

1052.6798017842948

In [34]:
mean_absolute_error(y_test, y_pred)

1052.6798017842948